## setup

In [2]:
import requests
from pprint import pprint,pformat

elucidate_base_url = 'http://localhost:8080/annotation'
headers = {
    'Accept':'application/ld+json; profile="http://www.w3.org/ns/anno.jsonld"',
    'Content-Type': 'application/ld+json; profile="http://www.w3.org/ns/anno.jsonld"'
}

def get_etag(url:str)->str:
    return requests.get(url).headers['etag'][3:-1]

## create container

In [3]:
body = {
  "@context": [
    "http://www.w3.org/ns/anno.jsonld",
    "http://www.w3.org/ns/ldp.jsonld"
  ],
  "type": [
    "BasicContainer",
    "AnnotationCollection"
  ],
  "label": "A Container for Web Annotations"
}

response = requests.post(url=elucidate_base_url+"/w3c/",headers=headers,json=body)
container_uri = response.json()['id']

response.status_code

201

## read container

In [4]:
requests.get(container_uri).json()

{'@context': ['http://www.w3.org/ns/anno.jsonld',
  'http://www.w3.org/ns/ldp.jsonld'],
 'id': 'http://localhost:8080/annotation/w3c/9c153658-6748-4fbe-bd41-c72554e67571/',
 'type': 'AnnotationCollection',
 'label': 'A Container for Web Annotations',
 'first': {'type': 'AnnotationPage',
  'as:items': {'@list': []},
  'partOf': 'http://localhost:8080/annotation/w3c/9c153658-6748-4fbe-bd41-c72554e67571/',
  'startIndex': 0},
 'last': 'http://localhost:8080/annotation/w3c/9c153658-6748-4fbe-bd41-c72554e67571/?page=0&desc=1',
 'total': 0}

## create annotation

In [5]:
annotation = {
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "type": "Annotation",
  "body": {
    "type": "TextualBody",
    "value": "I like this page!"
  },
  "target": "http://www.example.com/index.html"
}

response = requests.post(
    url=container_uri,
    headers=headers,
    json=annotation)
annotation_uri = response.json()['id']

response.status_code

201

response headers:

In [6]:
for (k,v) in sorted(response.headers.items()):
    print(f"{k} = {v}")

Allow = PUT,GET,OPTIONS,HEAD,DELETE
Cache-Control = no-cache, no-store, max-age=0, must-revalidate
Content-Type = application/ld+json; profile="http://www.w3.org/ns/anno.jsonld";charset=UTF-8
Date = Fri, 19 Mar 2021 10:20:09 GMT
ETag = W/"bd4f45eb4eccf1915511bc5bd29c298a"
Expires = 0
Link = <http://www.w3.org/ns/ldp#Resource>; rel="type"
Location = http://localhost:8080/annotation/w3c/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e
Memento-Datetime = Fri, 19 Mar 2021 10:20:09 UTC
Pragma = no-cache
Transfer-Encoding = chunked
Vary = Accept
X-Content-Type-Options = nosniff
X-Frame-Options = DENY
X-XSS-Protection = 1; mode=block


response body:

In [7]:
response.json()

{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'http://localhost:8080/annotation/w3c/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e',
 'type': 'Annotation',
 'body': {'type': 'TextualBody', 'value': 'I like this page!'},
 'target': 'http://www.example.com/index.html'}

## read annotation

- W3C :

In [8]:
pprint(annotation_uri)
requests.get(annotation_uri).json()

'http://localhost:8080/annotation/w3c/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e'


{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'http://localhost:8080/annotation/w3c/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e',
 'type': 'Annotation',
 'body': {'type': 'TextualBody', 'value': 'I like this page!'},
 'target': 'http://www.example.com/index.html'}

- OA :

In [9]:
oa_uri = annotation_uri.replace('/w3c/','/oa/')
pprint(oa_uri)
requests.get(oa_uri).json()

'http://localhost:8080/annotation/oa/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e'


{'@context': 'https://raw.githubusercontent.com/dlcs/elucidate-server/master/elucidate-server/src/main/resources/contexts/oa.jsonld',
 '@id': 'http://localhost:8080/annotation/oa/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e',
 '@type': 'oa:Annotation',
 'hasBody': {'@type': 'oa:TextualBody', 'value': 'I like this page!'},
 'hasTarget': 'http://www.example.com/index.html'}

## update annotation

In [10]:
etag = get_etag(annotation_uri)
new_annotation = {
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/w3c/mycontainer/anno1",
  "type": "Annotation",
  "created": "2015-01-31T12:03:45Z",
  "body": {
    "type": "TextualBody",
    "value": "I don't like this page!"
  },
  "target": "http://www.example.com/index.html"
}
update_headers = {'If-Match':etag}
update_headers.update(headers)
response = requests.put(url=annotation_uri,headers=update_headers,json=new_annotation)

response.status_code

200

In [11]:
requests.get(annotation_uri).json()

{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'http://localhost:8080/annotation/w3c/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e',
 'type': 'Annotation',
 'created': '2015-01-31T12:03:45Z',
 'body': {'type': 'TextualBody', 'value': "I don't like this page!"},
 'target': 'http://www.example.com/index.html'}

In [12]:
oa_uri = annotation_uri.replace('w3c','oa')
pprint(oa_uri)
requests.get(oa_uri).json()

'http://localhost:8080/annotation/oa/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e'


{'@context': 'https://raw.githubusercontent.com/dlcs/elucidate-server/master/elucidate-server/src/main/resources/contexts/oa.jsonld',
 '@id': 'http://localhost:8080/annotation/oa/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e',
 '@type': 'oa:Annotation',
 'annotatedAt': '2015-01-31T12:03:45Z',
 'hasBody': {'@type': 'oa:TextualBody', 'value': "I don't like this page!"},
 'hasTarget': 'http://www.example.com/index.html'}

## annotation history

In [13]:
response = requests.get(annotation_uri)
for (k,v) in sorted(response.headers.items()):
    print(f"{k} = {v}")

Allow = PUT,GET,OPTIONS,HEAD,DELETE
Cache-Control = no-cache, no-store, max-age=0, must-revalidate
Content-Type = application/ld+json;charset=UTF-8
Date = Fri, 19 Mar 2021 10:23:44 GMT
ETag = W/"e5784db0985b9c66e6f6142094edf460"
Expires = 0
Link = <http://www.w3.org/ns/ldp#Resource>; rel="type", <http://localhost:8080/annotation/w3c/services/history/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e/1>; rel="prev memento"; datetime="Fri, 19 Mar 2021 10:20:09 UTC"
Memento-Datetime = Fri, 19 Mar 2021 10:22:45 UTC
Pragma = no-cache
Transfer-Encoding = chunked
Vary = Accept
X-Content-Type-Options = nosniff
X-Frame-Options = DENY
X-XSS-Protection = 1; mode=block


In [14]:
response.headers['link']

'<http://www.w3.org/ns/ldp#Resource>; rel="type", <http://localhost:8080/annotation/w3c/services/history/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e/1>; rel="prev memento"; datetime="Fri, 19 Mar 2021 10:20:09 UTC"'

## delete annotation

In [15]:
etag = get_etag(annotation_uri)
delete_headers = {'If-Match':etag}
delete_headers.update(headers)
response = requests.delete(url=annotation_uri,headers=delete_headers)

response.status_code

204

In [16]:
requests.get(annotation_uri).status_code

410

In [18]:
response = requests.get("http://localhost:8080/annotation/w3c/services/history/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e/1")
response.status_code

404